In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [ ]:
np.random.seed(42)
BASE_PATH = "images"
RESIZED_PATH ="resized_images"

In [ ]:
images_folder = np.random.choice(os.listdir(BASE_PATH), 20)

images_path = []
images_size = []
    
for folder in images_folder:
    image_folder = os.path.join(BASE_PATH, folder)
    
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name).replace("\\","/")
        images_path.append(img_path)
        img = Image.open(img_path)
        images_size.append(img.size)
        
dataframe = pd.DataFrame(pd.Series(images_path).to_frame("path")) \
.join(pd.Series(images_size).to_frame("size"))

In [ ]:
dataframe['size_x'], dataframe['size_y'] = list(dataframe['size'].str)
dataframe.drop(['size'], axis=1, inplace=True)

In [ ]:
dataframe = dataframe[(dataframe['size_x'] > 100) & (dataframe['size_y'] > 100)]

In [ ]:
def crop_img(img, shape=(100,100)):
    width, height, _ = img.shape
    
    cx, cy = width / 2, height /2
    sx, sy = cx - shape[0] / 2, cy - shape[1] / 2
    ex, ey = cx + shape[0] / 2, cy + shape[1] / 2

    return img[int(sx) : int(ex), int(sy) : int(ey)]

In [ ]:
image_names = []

for index, row in tqdm(dataframe.iterrows()):
    path, size_x, size_y = row['path'], row['size_x'], row['size_y']
    img = cv2.imread(path)
    croped_img = crop_img(img)
    
    resized_path = os.path.join(RESIZED_PATH, f"img{index}.jpg")
    
    image_names.append(resized_path)
    cv2.imwrite(resized_path, croped_img)

In [ ]:
class DataGenerator(Sequence):
    
    def __init__(self, 
                 images_paths, 
                 seq = None,
                 batch_size = 32, 
                 image_dimensions = (100 ,100 ,3), 
                 shuffle = False):
        self.images_paths = images_paths
        self.seq = seq
        self.dim = image_dimensions
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

 

    def __len__(self):
        return int(np.floor(len(self.images_paths) / self.batch_size))

 

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.images_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

 

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        
        with mp.Pool() as pool:
            images = pool.map(cv2.imread, [self.images_paths[k] for k in indexes])

        images = np.array(images, np.float32) / 255
        
        return images